# 多クラス分類におけるROC/AUC

このノートブックでは、多クラス分類でのROC曲線とAUCの計算方法を学習します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

# 多クラス分類用データセット
X, y = make_classification(n_samples=1000, n_features=10, n_informative=8, n_classes=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# モデルの訓練
model = LogisticRegression(multi_class='multinomial', random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred_proba = model.predict_proba(X_test_scaled)

# ラベルの二値化
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])
n_classes = y_test_binarized.shape[1]

# 各クラスのROC曲線を描画
plt.figure(figsize=(12, 8))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {i} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curves (One-vs-Rest)')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

# Macro/Micro平均AUC
auc_macro = roc_auc_score(y_test_binarized, y_pred_proba, average='macro')
auc_micro = roc_auc_score(y_test_binarized, y_pred_proba, average='micro')

print('=== 多クラス分類でのAUC ===')
print(f'Macro-average AUC: {auc_macro:.4f}')
print(f'Micro-average AUC: {auc_micro:.4f}')
